# Łańcuch zobowiązań

Łańcuch Zobowiązań (ang. Chain of Responsibility) to czynnościowy wzorzec projektowy, który umożliwia przekazywanie żądania przez sekwencję potencjalnych obsługujących (handlerów) bez potrzeby jawnego określania, który z nich je obsłuży. Każdy handler decyduje, czy obsłużyć żądanie lub przekazać je dalej w łańcuchu. Dzięki takiemu podejściu możliwe jest dynamiczne kształtowanie procesu obsługi żądań i uniknięcie sztywnego powiązania między nadawcą a odbiorcą.

## Zastosowanie i przeznaczenie

- Eliminacja bezpośrednich powiązań między nadawcą, a odbiorcami żądania.
- Umożliwienie dynamicznego dodawania lub usuwania elementów obsługujących żądania.
- Zapewnienie elastyczności w obsłudze żądań poprzez kaskadowe delegowanie odpowiedzialności.

<img src="img/Chain_of_Responsibility_Design_Pattern_UML.jpg">

## Implementacja z wykorzystaniem listy hierarchii obiektów

Cel: symulacja procesu delegowania zadań w pewnej organizacji rozpoczynając od szczytu

In [ ]:
from abc import ABC

Abstrakcja klasy pracownika

In [ ]:
class TeamMember(ABC):
    task: str

    def process(self, task: str) -> bool | None:
        print(f"No one can do {task}. Please hire a new specialist.")

Klasa dyrektora, który w zakresie obowiązków ma metody: `meet_the_president()` oraz `meet_the_prime_minister()`

In [ ]:
class Director(TeamMember):
    def process(self, task: str) -> bool | None:
        if "meet" in task:
            if "president" in task:
                self.meet_the_president()
                return True
            elif "minister" in task:
                self.meet_the_prime_minister()
                return True
        
        return False

    @staticmethod
    def meet_the_president() -> None:
        print("Meet the president")

    @staticmethod
    def meet_the_prime_minister() -> None:
        print("Meet the Prime Minister")

Klasa zastępcy dyrektora, w zakresie obowiązków którego znajdują się metody: `meet_someone_else()` oraz `sign_documents()`.

In [ ]:
class ViceDirector(TeamMember):
    def process(self, task: str) -> bool | None:
        if "meet" in task:
            self.meet_someone_else()
            return True
        if "sign" in task:
            self.sign_documents()
        
        return False

    @staticmethod
    def meet_someone_else() -> None:
        print("Meet...")

    @staticmethod
    def sign_documents() -> None:
        print("Sign documents")

Klasa kierownika z metodami: `find_someone()`, `add_task()` oraz `check_the_process()`.

In [ ]:
class Manager(TeamMember):
    def process(self, task: str) -> bool | None:
        if "find" in task:
            self.find_someone()
            return True
        if "task" in task:
            self.add_task()
            return True
        if "check" in task and "process" in task:
            self.check_the_process()
            return True
        
        return False

    @staticmethod
    def find_someone() -> None:
        print("Find a new worker")

    @staticmethod
    def add_task() -> None:
        print("Add task to someone")

    @staticmethod
    def check_the_process() -> None:
        print("Check the process")

Klasa sekretarki z metodami: `call()`, `write_back()` oraz `get_tea()`.

In [ ]:
class Secretary(TeamMember):
    def process(self, task: str) -> bool | None:
        if "call" in task:
            self.call()
            return True
        if "write" in task and "back" in task:
            self.write_back()
            return True
        if "tea" in task:
            self.get_tea()
            return True

        return False

    @staticmethod
    def call() -> None:
        print("Call")

    @staticmethod
    def write_back() -> None:
        print("Write back")

    @staticmethod
    def get_tea() -> None:
        print("Get tea")

Klasa szeregowego pracownika, który ma jedynie metodę `do_work()`.

In [ ]:
class Worker(TeamMember):
    def process(self, task: str) -> bool | None:
        if "work" in task:
            self.do_work()
            return True

        return False

    @staticmethod
    def do_work():
        print("Do good job")


Łańcuch zobowiązań, który dopasowuje metodę do zadania wg jej nazwy. Hierarchia poszukiwań zaczyna się na szczycie i wędruje w dół.

In [ ]:
class Chain:
    chain: list

    def __init__(self):
        self.chain = []

    def run_task(self, task: str) -> None:
        for link in self.chain:
            result = link.process(task)
            if result:
                break

Kod klienta

In [ ]:
chain_0 = Chain()

In [ ]:
chain_0.chain.append(Director())
chain_0.chain.append(ViceDirector())
chain_0.chain.append(Manager())
chain_0.chain.append(Secretary())
chain_0.chain.append(Worker())
chain_0.chain.append(TeamMember())

In [ ]:
task = "bring a tea"
chain_0.run_task(task)

## Implementacja z wykorzystaniem zachowanej hierarchii

Cel: symulacja przechwytywania zdarzeń przez zagnieżdżone obiekty

In [ ]:
from abc import ABC
from typing import Self

Klasa reprezentują zdarzenie systemowe

In [ ]:
class Event:
    name: str

    def __init__(self, name: str) -> None:
        self.name = name

Abstrakcja widgetu mogącego obsłużyć wydarzenie.

In [ ]:
class Widget(ABC):
    parent: Self

    def __init__(self, parent: Self = None) -> None:
        self.parent = parent

    @staticmethod
    def default_handler(event: Event) -> None:
        print(f"Unable to handle event {event.name}")

    def handle(self, event: Event) -> None:
        handle_name = f"handle_{event.name}"
        if hasattr(self, handle_name):
            handler = getattr(self, handle_name)
            handler(event)
        elif self.parent is not None:
            self.parent.handle(event)
        else:
            self.default_handler(event)

Klasa widgetu okna, które obsługuje wydarzenie `close`.

In [ ]:
class Window(Widget):
    @staticmethod
    def handle_close(event: Event) -> None:
        print(f"Window closed by event: {event}")

Klasa widgetu okna dialogowego obsługującego zdarzenia: `ok`, `cancel` i `send`.

In [ ]:
class Dialog(Widget):
    @staticmethod
    def handle_ok(event: Event) -> None:
        print(f"Accepted by event: {event}")

    @staticmethod
    def handle_cancel(event: Event) -> None:
        print(f"Cancel by event: {event}")

    @staticmethod
    def handle_send(event: Event) -> None:
        print(f"Sent by event: {event}")

Kod klienta

In [ ]:
window = Window()
dialog = Dialog(parent=window)

In [ ]:
event = Event("ok")
dialog.handle(event)

In [ ]:
event = Event("close")
dialog.handle(event)

In [ ]:
event = Event("closaae")
dialog.handle(event)

## Podsumowanie

Łańcuch Zobowiązań to czynnościowy wzorzec projektowy, który umożliwia przekazywanie żądania przez sekwencję potencjalnych obsługujących (handlerów) bez potrzeby jawnego określania, który z nich je obsłuży. Takie podejście rodzi konsekwencje:
- ignorowanie lub oddelegowywanie zadań dalej w określonej hierarchii,
- ogniwa łańcucha mozna modyfikować w sposób dynamiczny,
- zadanie zostanie wykonane przez pierwszy obiekt w hierarchii potrafiący je obsłużyć,
- mogą istnieć zadania nieobsłużone gdy nie istnieje domyślny handler,
- każdy element łancucha powinien znać co najwyżej 2 sąsiednie elementy.